In [ ]:

pip install "altair[all]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.4 MB/s eta 0:00:00


In [ ]:

df = pd.read_csv("Datos_visualizacion.csv", sep=";")


df["Año de estreno"] = pd.to_numeric(df["Año de estreno"], errors="coerce")

df["Contenido"] = (
    df["Contenido"]
    .str.strip()

)
conteo = (
    df.groupby(["Año de estreno", "Contenido"])
    .size()
    .reset_index(name="Cantidad")
)

chart = (
    alt.Chart(conteo)
    .mark_line(point=True, interpolate='monotone')
    .encode(
        x=alt.X("Año de estreno:O", title="Año de estreno"),
        y=alt.Y("Cantidad:Q", title="Cantidad de películas"),
        color=alt.Color("Contenido:N", title="Tipo de contenido", scale=alt.Scale(scheme="tableau10")),
        tooltip=["Año de estreno", "Contenido", "Cantidad"]
    )
    .properties(
        title="Evolución de los tipos de contenido en películas Disney",
        width=800,
        height=400
    )
    .configure_title(fontSize=18, anchor="start", color="gray")
    .configure_axis(labelFontSize=12, titleFontSize=14)
    .configure_legend(titleFontSize=13, labelFontSize=12)
)

chart

alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import os
from IPython.display import HTML, display

fn = "Datos_visualizacion.csv"
df = pd.read_csv(fn, sep=";", encoding="utf-8", dtype=str)

df.columns = [c.strip() for c in df.columns]

col_map = {
    "Título en español": "titulo",
    "Tipo de película": "tipo",
    "Año de estreno": "anio",
    "Contenido": "contenido"
}
df = df.rename(columns={k: v for k, v in col_map.items() if k in df.columns})

for c in ["titulo", "tipo", "contenido", "anio"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()

df["contenido_clean"] = df.get("contenido", "").astype(str).str.lower().str.strip()
df["categoria"] = df["contenido_clean"].apply(lambda x: "Originales" if x == "original" else "Otras")

df["anio_num"] = df.get("anio", "").astype(str).str.extract(r"(\d{4})")[0]
df["anio_num"] = pd.to_numeric(df["anio_num"], errors="coerce")
df = df.dropna(subset=["anio_num"])
df["anio_num"] = df["anio_num"].astype(int)
df["anio_dt"] = pd.to_datetime(df["anio_num"], format="%Y")

if df.empty or df.shape[0] == 0:
    print("ERROR: el DataFrame quedó vacío después de la limpieza. Aquí las primeras filas crudas:")
    print(pd.read_csv(fn, sep=";", encoding="utf-8", dtype=str).head(10))
    raise SystemExit("Terminado por DataFrame vacío")

df_count = (
    df.groupby(["anio_dt", "categoria"], as_index=False)
      .size()
      .rename(columns={"size": "cantidad"})
)

print("Preview de los datos agrupados (df_count.head()):")
print(df_count.head(10))
chart = (
    alt.Chart(df_count)
    .mark_area(interpolate="monotone")
    .encode(
        x=alt.X("anio_dt:T", title="Año"),
        y=alt.Y("cantidad:Q", stack="zero", title="Cantidad de títulos"),
        color=alt.Color("categoria:N", title="Tipo"),
        tooltip=[
            alt.Tooltip("anio_dt:T", title="Año"),
            alt.Tooltip("categoria:N", title="Categoría"),
            alt.Tooltip("cantidad:Q", title="Cantidad")
        ]
    )
    .properties(width=850, height=450, title="Títulos por año — Originales vs Otras")
)

try:
    alt.renderers.enable('html')
except Exception:
    pass

try:
    html = chart.to_html()
    display(HTML(html))
    print("Se intentó mostrar el gráfico en el notebook usando chart.to_html().")
except Exception as e:
    print("No se pudo mostrar con chart.to_html():", e)

out_fname = "grafico_area.html"
try:
    chart.save(out_fname)
    print(f"Gráfico guardado en: {os.path.abspath(out_fname)}")
    print("Ábrelo en tu navegador (doble clic) si no se renderiza aquí.")
except Exception as e:

    try:
        with open(out_fname, "w", encoding="utf-8") as f:
            f.write(chart.to_html())
        print(f"(Fallback) Gráfico escrito en: {os.path.abspath(out_fname)}")
    except Exception as e2:
        print("No se pudo guardar el HTML automáticamente:", e, e2)




Preview de los datos agrupados (df_count.head()):
     anio_dt   categoria  cantidad
0 1995-01-01  Originales         9
1 1995-01-01       Otras         2
2 1996-01-01  Originales         3
3 1996-01-01       Otras         5
4 1997-01-01  Originales         4
5 1997-01-01       Otras         5
6 1998-01-01  Originales         4
7 1998-01-01       Otras         6
8 1999-01-01  Originales         3
9 1999-01-01       Otras         6


Se intentó mostrar el gráfico en el notebook usando chart.to_html().
Gráfico guardado en: /content/grafico_area.html
Ábrelo en tu navegador (doble clic) si no se renderiza aquí.
